In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Activation, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from qkeras import *
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

2022-11-17 01:33:10.912865: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 01:33:11.060062: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-17 01:33:11.060098: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-17 01:33:11.083816: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 01:33:11.725993: W tensorflow/stream_executor/platform/de

In [2]:
train_df = pd.read_csv('train_1Dy.csv',nrows=1000)
test_df = pd.read_csv('test_1Dy.csv',nrows=500)

In [3]:
X_train = train_df.drop(columns=['y-midplane','cotBeta']).values
X_test = test_df.drop(columns=['y-midplane','cotBeta']).values

y_train = train_df['y-midplane'].values
y_test = test_df['y-midplane'].values

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.fit_transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [5]:
maxval = 1e9
minval = 1e-9

pi = 3.14159265359
sqrtPiOn2 = 1.25331413732
sqrt2 = 1.41421356237

def double_crystalball_norm(mu, sigma, alphaL, nL, alphaR, nR):
    
    LN_top = tf.math.exp(-0.5*alphaL*alphaL)*nL
    LN_bottom = alphaL*(nL-1.0)

    RN_top = tf.math.exp(-0.5*alphaR*alphaR)*nR
    RN_bottom = alphaR*(nR-1.0)

    CN = sqrtPiOn2 * (tf.math.erf(alphaL/sqrt2) + tf.math.erf(alphaR/sqrt2))

    den = (1.0*LN_top/LN_bottom + 1.0*RN_top/RN_bottom + CN) * sigma
    
    return 1.0/den

def dcsb_simon(y, p):
       
    mu     = tf.gather(params=p, indices=[0], axis=1)
    sigma  = 0.1 + tf.math.softplus(tf.gather(params=p, indices=[1], axis=1))
    alphaR = 1.1 + tf.math.softplus(tf.gather(params=p, indices=[2], axis=1))
    nR     = 1.1 + tf.math.softplus(tf.gather(params=p, indices=[3], axis=1))
    alphaL = 1.1 + tf.math.softplus(tf.gather(params=p, indices=[4], axis=1))
    nL     = 1.1 + tf.math.softplus(tf.gather(params=p, indices=[5], axis=1))
    
    #tf.print(mu,sigma,alphaR,nR,alphaL,nL)
    
    t = 1.0*(y-mu)/sigma
    #tf.print('t',t) 

    # start with Gaussian likelihood
    likelihood = tf.math.exp(-0.5*t*t)

    # Replace low tail
    factL = 1 - 1.0*alphaL*alphaL/nL - 1.0*t*alphaL/nL
    factL = tf.where(factL<0, minval, factL)
    termLeft = tf.math.exp(-0.5*alphaL*alphaL) * tf.math.pow(factL, -1.0*nL)
    #tf.print('factl',factL)
    #tf.print('termleft',termLeft)
    likelihood = tf.where(t<-1*alphaL, termLeft, likelihood)
    
    # Replace high tail
    factR = 1 - 1.0*alphaR*alphaR/nR + 1.0*t*alphaR/nR
    factR = tf.where(factR<0, minval, factR)
    termRight = tf.math.exp(-0.5*alphaR*alphaR) * tf.math.pow(factR, -1.0*nR)
    #tf.print('factr',factR)
    #tf.print('termright',termRight)
    likelihood = tf.where(t>alphaR, termRight, likelihood)
    
    #tf.print('likelihood',likelihood)

    norm = double_crystalball_norm(mu, sigma, alphaL, nL, alphaR, nR)  
    likelihood = norm*likelihood
    #tf.print('norm',norm)
    
    # Removing zero likelihood
    likelihood = tf.clip_by_value(likelihood,minval,maxval)
    
    NLL = -1.0*tf.math.log(likelihood)
    #tf.print(NLL)
    
    return tf.keras.backend.sum(NLL,axis=1)

In [6]:
def CreateModel(shape):
    x = x_in = Input(shape)
    x = Dense(64, name="dense1")(x)
    x = Activation("relu", name="relu1")(x)
    #x = BatchNormalization()(x)
    x = Dense(6, name="dense2")(x)
    model = Model(inputs=x_in, outputs=x)
    return model

model=CreateModel(X_train.shape[1:])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss=dcsb_simon)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense1 (Dense)              (None, 64)                896       
                                                                 
 relu1 (Activation)          (None, 64)                0         
                                                                 
 dense2 (Dense)              (None, 6)                 390       
                                                                 
Total params: 1,286
Trainable params: 1,286
Non-trainable params: 0
_________________________________________________________________


2022-11-17 01:33:13.478918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 01:33:13.479352: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-17 01:33:13.479425: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-17 01:33:13.479488: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-17 01:33:13.479550: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [ ]:
es = EarlyStopping(patience=10,
                  restore_best_weights=True)

history = model.fit(X_train,
                    y_train,
                    callbacks=[es],
                    epochs=5, 
                    batch_size=1,
                    validation_split=0.2,
                    shuffle=True,
                    verbose=1)

Epoch 1/5
[[5.86624098]]
  1/800 [..............................] - ETA: 18:21 - loss: 5.8662[[7.12489271]]
[[7.0485034]]
[[5.04963112]]
[[10.0660648]]
[[6.59569454]]
[[6.45337439]]
[[10.5295525]]
[[7.08610773]]
[[5.95926571]]
[[4.63077307]]
[[7.12125111]]
[[4.42224503]]
[[6.04410791]]
[[7.6361928]]
[[6.96571207]]
[[7.2936368]]
[[6.24459076]]
[[5.47582865]]
[[7.38854933]]
 20/800 [..............................] - ETA: 2s - loss: 6.7501   [[8.00028896]]
[[5.67796087]]
[[3.32702041]]
[[3.78348136]]
[[8.0445528]]
[[7.8217783]]
[[5.01027393]]
[[7.51865721]]
[[7.06387615]]
[[1.07639313]]
[[7.38451767]]
[[7.53449297]]
[[8.02706146]]
[[7.37274361]]
[[7.88376617]]
[[7.09156036]]
[[7.1596489]]
[[10.6583815]]
[[3.43055844]]
[[6.42083168]]
[[7.46882296]]
[[7.42149782]]
 42/800 [>.............................] - ETA: 1s - loss: 6.6710[[1.01784074]]
[[6.37977791]]
[[6.98697]]
[[9.01562595]]
[[1.11043847]]
[[7.98324299]]
[[6.78018379]]
[[7.5756259]]
[[7.78907347]]
[[6.84231138]]
[[6.61091805]]
[[7.

In [ ]:
history_dict = history.history
loss_values = history_dict['loss'] 
val_loss_values = history_dict['val_loss'] 
epochs = range(1, len(loss_values) + 1) 
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'orange', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.yscale('Log')
plt.legend()

In [ ]:
#model.save('models_1Dy/gausstest.keras')

In [ ]:
p_test = model.predict(X_test)
df = pd.DataFrame(p_test,columns=['mu','sigma','alphaR','alphaL','nR','nL'])
df['sigma'] = 0.1 + tf.math.softplus(df['sigma'])
df['alphaR'] = 0.1 + tf.math.softplus(df['alphaR'])
df['alphaL'] = 0.1 + tf.math.softplus(df['alphaL'])
df['nR'] = 0.1 + tf.math.softplus(df['nR'])
df['nL'] = 0.1 + tf.math.softplus(df['nL'])
df['ytrue'] = y_test
df.to_csv("test_1DyDCSB.csv",header=False,index=False)

In [ ]:
residuals = y_test - p_test[:,0]

In [ ]:
plt.hist(residuals,bins=50)
plt.xlabel(r'$y-\mu_y$ [um]')
plt.show()

plt.hist(df['sigma'],bins=50);
plt.xlabel(r'$\sigma_y$ [um]')
plt.show()

plt.hist(df['alphaR'],bins=50);
plt.xlabel(r'$\alpha_R$')
plt.show()

plt.hist(df['alphaL'],bins=50);
plt.xlabel(r'$\alpha_L$')
plt.show()

plt.hist(df['nR'],bins=50);
plt.xlabel(r'$n_R$')
plt.show()

plt.hist(df['nL'],bins=50);
plt.xlabel(r'$n_L$')
plt.show()

In [ ]:
plt.errorbar(x=y_test,y=p_test[:,0],yerr=p_test[:,1],linestyle='',marker='.',markerfacecolor='black')
plt.xlabel(r'$y$ [um]')
plt.ylabel(r'Predicted $y$ [um]')

In [ ]:
plt.scatter(y_test,p_test[:,1])
plt.xlabel(r'$y$ [um]')
plt.ylabel(r'$\sigma_y$ [um]')

In [ ]:
pulls = residuals/p_test[:,1]

In [ ]:
plt.hist(pulls,bins=50);
plt.xlabel('pull')

In [ ]:
print(np.mean(pulls),np.std(pulls))

In [ ]:
y_train

In [ ]:
tf.print(tf.math.pow(0.,-1))

In [ ]:
# old, ignore
def dcsb(y, p):
       
    mu     = tf.gather(params=p, indices=[0], axis=1)
    sigma  = 0.1 + tf.math.softplus(tf.gather(params=p, indices=[1], axis=1))
    alphaR = 1.1 + tf.math.softplus(tf.gather(params=p, indices=[2], axis=1))
    nR     = 1.1 + tf.math.softplus(tf.gather(params=p, indices=[3], axis=1))
    alphaL = 1.1 + tf.math.softplus(tf.gather(params=p, indices=[4], axis=1))
    nL     = 1.1 + tf.math.softplus(tf.gather(params=p, indices=[5], axis=1))
    
    tf.print(mu,sigma,alphaR,nR,alphaL,nL)
    
    t = 1.0*(y-mu)/sigma

    term1Left  = 1 - alphaL*alphaL/nL - alphaL*t/nL
    normLeft   = tf.math.exp(-0.5*alphaL*alphaL)
    term1Right = 1 - alphaR*alphaR/nR + alphaR*t/nR
    normRight  = tf.math.exp(-0.5*alphaR*alphaR)
    
    # pow function cares if you ask it to do 0 to a negative power
    termLeft  = normLeft*tf.math.pow(term1Left,-1*nL)
    termRight = normRight*tf.math.pow(term1Right,-1*nR)
    termMiddle = tf.math.exp(-0.5*t*t)
    
    tf.print("t",t)
    #tf.print("right",termRight)
    
    # Piecewise elements are zero outside their validity range
    # likely not a problem? try regular multiply
    termLeft = tf.where(t>=-1*alphaL,0.0,termLeft)
    termRight = tf.where(t<=alphaR,0.0,termRight)
    termMiddle = tf.where(t<-1*alphaL,0.0,termMiddle)
    termMiddle = tf.where(t>alphaR,0.0,termMiddle)
    
    tf.print("left again",termLeft)
    tf.print("right again", termRight)
    tf.print("middle",termMiddle)
    
    #termLeft   = tf.math.multiply_no_nan(tf.cast(t<-1*alphaL,tf.float32),termLeft)
    #termRight  = tf.math.multiply_no_nan(tf.cast(t>alphaR,tf.float32),termRight)
    #termMiddle = tf.math.multiply_no_nan(tf.cast(t>=-1*alphaL,tf.float32)*tf.cast(t<=alphaR,tf.float32),termMiddle)
    
    # Add the piecewise terms together with normalization
    norm = double_crystalball_norm(mu, sigma, alphaL, nL, alphaR, nR)
    tf.print("norm",norm)

    likelihood = norm*(termLeft + termMiddle + termRight)
    
    # Removing zero likelihood
    likelihood = tf.clip_by_value(likelihood,minval,maxval)
    
    NLL = -1.0*tf.math.log(likelihood)
    tf.print(NLL)
    
    return tf.keras.backend.sum(NLL,axis=1)

In [ ]:
math.exp(0.5*-14.3461494*-14.3461494)